## TRADE DATA

In [1]:
import pandas as pd
import datetime
import pandas as pd
import datetime
import numpy as np
import openpyxl

def process_csv(filepath, product_index):
    """
    This function takes the file path and product index as inputs and returns the filtered trade data dataframe
    for a particular product.
    
    Parameters:
    filepath (str) : file path for the trade data file
    product_index (int) : index of the product in the list of all products
    
    Returns:
    df_product_filtered (pandas dataframe) : dataframe for the filtered product
    """
    
    # Read the CSV file and convert the "TIME_PERIOD" column to datetime
    trade_data = pd.read_csv(filepath)
    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    
    # Rename columns and remove duplicates
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    
    # Get the list of all products
    products = trade_data["product"].drop_duplicates().to_list()
    
    # Filter the data for a particular product
    product_filter = products[product_index]
    
    # Read the partner countries data
    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")
    
    # Merge the trade data and partner country data on the "PARTNER_codes" column
    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')
    
    # Pivot the trade data to create a multi-index dataframe
    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'], 
                                  columns='indicators', 
                                  values=['OBS_VALUE'])
    
    # Flatten the column names
    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    
    # Reset the index and remove missing values
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()
    
    # Filter the data for the selected product
    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]
    

    return df_product_filtered

#Declare DataFrames 
df_maize_corn_0 = process_csv("trade_data/maize_corn.csv", 0)
df_maize_corn_0['price'] = df_maize_corn_0['OBS_VALUE_VALUE_1000EURO'] / df_maize_corn_0['OBS_VALUE_QUANTITY_TON']

# Keep '10059000' the same but name it 'maize'
df_maize = df_maize_corn_0
df_maize['product'] = 'maize'

# group by TIME_PERIOD and aggregate the OBS_VALUE_QUANTITY_TON and OBS_VALUE_VALUE_1000EURO columns by summing
grouped_df = df_maize.groupby('TIME_PERIOD').agg({'OBS_VALUE_QUANTITY_TON': 'sum', 'OBS_VALUE_VALUE_1000EURO': 'sum'})

# calculate globalprice
globalprice = pd.DataFrame(grouped_df['OBS_VALUE_VALUE_1000EURO'] / grouped_df['OBS_VALUE_QUANTITY_TON'])
# rename the column to 'globalprice'
globalprice = globalprice.rename(columns={0: 'globalprice'})


globalprice = globalprice.reset_index()
globalprice = globalprice.rename(columns={'index': 'TIME_PERIOD'})


globalprice
# add globalprice as a new column to the dataframe
#df_maize = df_maize.merge(globalprice, on=['TIME_PERIOD'], how='outer')

#df_maize


,TIME_PERIOD,globalprice
0,2005-01-01,0.122419
1,2005-02-01,0.118578
2,2005-03-01,0.121397
3,2005-04-01,0.116070
4,2005-05-01,0.122371
...,...,...
209,2022-06-01,0.362607
210,2022-07-01,0.403995
211,2022-08-01,0.254172
212,2022-09-01,0.310498


In [2]:
df_maize.head(2050)

,PARTNER_Labels,TIME_PERIOD,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,Afghanistan,2021-03-01,maize,0.0,0.04,inf
1,Antigua and Barbuda,2020-01-01,maize,54.0,21.20,0.392593
2,Antigua and Barbuda,2020-04-01,maize,27.0,12.14,0.449630
3,Argentina,2005-01-01,maize,8100.0,937.50,0.115741
4,Argentina,2005-02-01,maize,3172.0,392.55,0.123755
...,...,...,...,...,...,...
2045,Hungary,2006-05-01,maize,19745.2,2479.14,0.125557
2046,Hungary,2006-06-01,maize,18269.8,2330.37,0.127553
2047,Hungary,2006-07-01,maize,7446.7,1068.43,0.143477
2048,Hungary,2006-08-01,maize,1524.2,244.47,0.160392


## WEATHER DATA

In [3]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_global_temperature = pd.read_csv('weather_data/global_temperature_data.csv')

# Rename the 'YearMonth' column to 'TIME_PERIOD'
df_global_temperature.rename(columns={'yearmonth': 'TIME_PERIOD', 'partner_labels': 'PARTNER_Labels'}, inplace=True)

# Convert the TIME_PERIOD column to datetime format
df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'], format='%Y%m')

# Convert the TIME_PERIOD column to the desired format
df_global_temperature['TIME_PERIOD'] = df_global_temperature['TIME_PERIOD'].dt.strftime('%Y-%m-%d')

# Drop columns
df_global_temperature.drop(columns=['partner_code', 'weather_countrycode', 'weather_countryname', 'iso_countrycode'], inplace=True)

df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'])
df_global_temperature

,PARTNER_Labels,TIME_PERIOD,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,United Arab Emirates,2005-01-01,186.096154,110.000000,280.000000
1,United Arab Emirates,2005-02-01,204.750000,106.000000,306.500000
2,United Arab Emirates,2005-03-01,238.226667,124.500000,363.666667
3,United Arab Emirates,2005-04-01,280.984568,155.000000,423.333333
4,United Arab Emirates,2005-05-01,321.296296,192.000000,435.500000
...,...,...,...,...,...
21085,Viet Nam,2022-08-01,287.060676,244.363636,348.666667
21086,Viet Nam,2022-09-01,279.751352,233.600000,350.000000
21087,Viet Nam,2022-10-01,246.000000,238.000000,260.000000
21088,Viet Nam,2022-11-01,272.000000,252.000000,304.000000


## OIL DATA

In [4]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the 'oil_price_index_2015_oecd.xlsx' file
df_oil_index = pd.read_excel('oil_prices/oil_price_index_2015_oecd.xlsx', skiprows=[0,1,2,3,4, 9012, 9013, 9014, 9015])
# Forward fill the 'Country' column
df_oil_index['Country'].fillna(method='ffill', inplace=True)
# Rename columns
df_oil_index.rename(columns={'Unnamed: 3': 'CPI_ENERGY'}, inplace=True)
df_oil_index.rename(columns={'Unnamed: 4': 'CPI_ENERGY_HARMONISED'}, inplace=True)
# Delete the 'Unnamed: 2' column
df_oil_index.drop(columns=['Unnamed: 2'], inplace=True)
# Convert the 'Time' column to datetime format
df_oil_index['Time'] = pd.to_datetime(df_oil_index['Time'], format='%b-%Y')
df_oil_index.rename(columns={'Time': 'TIME_INDEX'}, inplace=True)

#Clean import CMO-Historical-Data-Monthly.xlsx

df_commodities_price = pd.read_excel('oil_prices/CMO-Historical-Data-Monthly.xlsx', sheet_name = 'Monthly Prices', skiprows=[0,1,2,3,4,5])
# Rename columns
df_commodities_price.rename(columns={'Unnamed: 0': 'TIME_PERIOD'}, inplace=True)
df_commodities_price.rename(columns={'CRUDE_PETRO': 'OIL_PRICE $/bbl'}, inplace=True)
df_commodities_price['TIME_PERIOD'] = pd.to_datetime(df_commodities_price['TIME_PERIOD'], format='%YM%m')

df_oil_price_full = pd.DataFrame(df_commodities_price, columns=['TIME_PERIOD', 'OIL_PRICE $/bbl'])

df_oil_price = df_oil_price_full.loc[(df_oil_price_full['TIME_PERIOD'] >= '2005-01-01') & (df_oil_price_full['TIME_PERIOD'] <= '2022-10-01')]
df_oil_price

,TIME_PERIOD,OIL_PRICE $/bbl
540,2005-01-01,42.972278
541,2005-02-01,44.818211
542,2005-03-01,50.942879
543,2005-04-01,50.640476
544,2005-05-01,47.826573
...,...,...
749,2022-06-01,116.800000
750,2022-07-01,105.083333
751,2022-08-01,95.973333
752,2022-09-01,88.220000


## FUTURES PRICES (CORN)

In [2]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_futures_corn = pd.read_csv('futures_data/US Corn Futures Historical Data.csv')

# Rename the 'date' column to 'TIME_PERIOD'
df_futures_corn.rename(columns={'Date': 'TIME_PERIOD', 'Price': 'Corn Price Futures'}, inplace=True)

df_futures_corn['TIME_PERIOD'] = pd.to_datetime(df_futures_corn['TIME_PERIOD'], format='%d/%m/%Y')
df_futures_corn['TIME_PERIOD'] = df_futures_corn['TIME_PERIOD'].dt.strftime('%Y/%d/%m')

df_futures_corn.drop(columns=['Open','High','Low','Vol.','Change %'], inplace=True)

df_futures_corn['TIME_PERIOD'] = pd.to_datetime(df_futures_corn['TIME_PERIOD'])

df_futures_corn.sort_values(by='TIME_PERIOD', ascending=True, inplace=True)
df_futures_corn['TIME_PERIOD']

213   2005-01-01
212   2005-02-01
211   2005-03-01
210   2005-04-01
209   2005-05-01
         ...    
4     2022-06-01
3     2022-07-01
2     2022-08-01
1     2022-09-01
0     2022-10-01
Name: TIME_PERIOD, Length: 214, dtype: datetime64[ns]

## MACROECONOMIC DATA

In [6]:
#import datetime
import numpy
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

path_file_1 = "macroeconomic_data/macro_economic_indicators.csv"
economic_indicators = pd.read_csv(path_file_1, sep = ";", decimal=".")

#rename column for merge later
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels'})
economic_indicators = economic_indicators.rename(columns = {'country_name': 'PARTNER_Labels' ,'Time':'TIME_PERIOD' })
#this is a quick fix
macro_economic_indicators = economic_indicators
# Convert year to datetime with year and month
macro_economic_indicators['year'] = pd.to_datetime(macro_economic_indicators['year'], format='%Y').dt.to_period('M')
# Set the date as the index
macro_economic_indicators.set_index('year', inplace=True)

#make df for monthly data
monthly_data = macro_economic_indicators.iloc[0:0].copy()


number_of_col = macro_economic_indicators.shape[1]
nan_row = numpy.empty((number_of_col,))
nan_row[:] = numpy.nan

#add na rows for 11 months in a year
n_row = macro_economic_indicators.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = macro_economic_indicators.iloc[i,:]
    current_country = macro_economic_indicators.iloc[i,0] 
    current_country_code = macro_economic_indicators.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data.loc[index_3] = holder_for_rows   
        else:
            monthly_data.loc[index_3] = nan_row
            monthly_data.iloc[index_3,0] = current_country
            monthly_data.iloc[index_3,1] = current_country_code

#slects als the column with numbers
c = monthly_data.columns[3:12]
monthly_data[c] = monthly_data[c].apply(pd.to_numeric)

n_countries = len(population_data['Country Code'].unique())
#fills in the date column
for i in range(n_countries-1):
    index_1 = 0   + 204*i
    index_2 = 204 + 204*i
    monthly_data.iloc[index_1:index_2,:] = monthly_data.iloc[index_1:index_2,:].interpolate().copy()
    #adding dates 
    monthly_data.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2021-12", freq='M')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700


In [7]:
import pandas as pd
import datetime
import numpy as np
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

#delete the last 5 empty rows 
num_rows = len(population_data)
population_data = population_data.drop(population_data.index[num_rows-5:num_rows])
#rename a column 
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels','Time':'TIME_PERIOD'})
population_data = population_data.rename(columns = {'Population, total [SP.POP.TOTL]': 'A'})

#get's the index s of the columns with string
df = population_data
string_indices = df['A'][df['A'].apply(lambda x: isinstance(x, str))].index

#gets th countries corresponging to the index s 
lll = len(string_indices)
LIST_of_del_countries = [None] * lll

counter = 0 
for i in range(lll):
    LIST_of_del_countries[counter] = population_data.iloc[(string_indices[i]),0]
    counter = counter +1

list1 = list(set(LIST_of_del_countries))

print(df.shape)
print(list1)
for j in range(4000):
  
    if df.iloc[j,0] in list1:
        df = df.drop(index=[j])
        print(df.shape)

df = df.replace('..', 0)
print(df)
# boolean indexing to select rows to delete
idx = df['PARTNER_Labels'].str.contains('IDA blend')

# drop rows using boolean indexing
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA only')
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA total')
df = df.drop(df[idx].index)

for name in list1:
    idx = df['PARTNER_Labels'].str.contains(name)
    df = df.drop(df[idx].index)


testing= df.copy()
excel_test = pd.ExcelWriter("population_data.xlsx")
testing.to_excel(excel_test)
excel_test.save()
#make the 
monthly_data_p = population_data.iloc[0:0].copy()

number_of_col = df.shape[1]
nan_row = np.empty((number_of_col,))
nan_row[:] = np.nan

n_row = df.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = df.iloc[i,:]
    current_country = df.iloc[i,0]
    current_country_code = df.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data_p.loc[index_3] = holder_for_rows   
        else:
            monthly_data_p.loc[index_3] = nan_row
            monthly_data_p.iloc[index_3,0] = current_country
            monthly_data_p.iloc[index_3,1] = current_country_code 


# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper2.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()

c_p = population_data.columns[4]
monthly_data_p[c_p] = monthly_data_p[c_p].apply(pd.to_numeric)
n_countries = len(df['Country Code'].unique())
#fills in the date column
for i in range(n_countries-10):
    index_1 = 0   + 228*i
    index_2 = 228 + 228*i
    monthly_data_p.iloc[index_1:index_2,4] = monthly_data_p.iloc[index_1:index_2,4].interpolate().copy()
    #adding dates 
    monthly_data_p.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2023-12", freq='M')

# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper3.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()


(5054, 5)
['IDA blend', 'Bermuda', 'IBRD only', 'Not classified', 'St. Martin (French part)', 'Sint Maarten (Dutch part)', 'IDA only', 'IDA total', 'Greenland']
(5053, 5)
(5052, 5)
(5051, 5)
(5050, 5)
(5049, 5)
(5048, 5)
(5047, 5)
(5046, 5)
(5045, 5)
(5044, 5)
(5043, 5)
(5042, 5)
(5041, 5)
(5040, 5)
(5039, 5)
(5038, 5)
(5037, 5)
(5036, 5)
(5035, 5)
(5034, 5)
(5033, 5)
(5032, 5)
(5031, 5)
(5030, 5)
(5029, 5)
(5028, 5)
(5027, 5)
(5026, 5)
(5025, 5)
(5024, 5)
(5023, 5)
(5022, 5)
(5021, 5)
(5020, 5)
(5019, 5)
(5018, 5)
(5017, 5)
(5016, 5)
(5015, 5)
(5014, 5)
(5013, 5)
(5012, 5)
(5011, 5)
(5010, 5)
(5009, 5)
(5008, 5)
(5007, 5)
(5006, 5)
(5005, 5)
(5004, 5)
(5003, 5)
(5002, 5)
(5001, 5)
(5000, 5)
(4999, 5)
(4998, 5)
(4997, 5)
(4996, 5)
(4995, 5)
(4994, 5)
(4993, 5)
(4992, 5)
(4991, 5)
(4990, 5)
(4989, 5)
(4988, 5)
(4987, 5)
(4986, 5)
(4985, 5)
(4984, 5)
(4983, 5)
(4982, 5)
(4981, 5)
(4980, 5)
(4979, 5)
(4978, 5)
(4977, 5)
(4976, 5)
(4975, 5)
     PARTNER_Labels Country Code  TIME_PERIOD Tim

C:\Users\basbe\AppData\Local\Temp\ipykernel_20852\3099761979.py:54: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  idx = df['PARTNER_Labels'].str.contains(name)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063


3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705


In [8]:
monthly_data_p.head(214)

,PARTNER_Labels,Country Code,TIME_PERIOD,Time Code,A
0,Afghanistan,AFG,2005-01,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05,NaN,2.475511e+07
...,...,...,...,...,...
209,Afghanistan,AFG,2022-06,NaN,4.159172e+07
210,Afghanistan,AFG,2022-07,NaN,4.168431e+07
211,Afghanistan,AFG,2022-08,NaN,4.177690e+07
212,Afghanistan,AFG,2022-09,NaN,4.186949e+07


In [9]:
monthly_data_p  = monthly_data_p.rename(columns = {'Country Code': 'country_code'})
# convert Period and float values to string format, and then to datetime
monthly_data_p ['TIME_PERIOD'] = monthly_data_p ['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data_p ['TIME_PERIOD'] = pd.to_datetime(monthly_data_p ['TIME_PERIOD'], errors='coerce')
monthly_data_p.head(214)

,PARTNER_Labels,country_code,TIME_PERIOD,Time Code,A
0,Afghanistan,AFG,2005-01-01,YR2005,2.441119e+07
1,Afghanistan,AFG,2005-02-01,NaN,2.449717e+07
2,Afghanistan,AFG,2005-03-01,NaN,2.458315e+07
3,Afghanistan,AFG,2005-04-01,NaN,2.466913e+07
4,Afghanistan,AFG,2005-05-01,NaN,2.475511e+07
...,...,...,...,...,...
209,Afghanistan,AFG,2022-06-01,NaN,4.159172e+07
210,Afghanistan,AFG,2022-07-01,NaN,4.168431e+07
211,Afghanistan,AFG,2022-08-01,NaN,4.177690e+07
212,Afghanistan,AFG,2022-09-01,NaN,4.186949e+07


In [10]:
monthly_data  = monthly_data.rename(columns = {'timecode': 'TIME_PERIOD'})
# convert Period and float values to string format, and then to datetime
monthly_data['TIME_PERIOD'] = monthly_data['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data['TIME_PERIOD'] = pd.to_datetime(monthly_data['TIME_PERIOD'], errors='coerce')

In [11]:
merged_monthly_data_df = monthly_data.merge(monthly_data_p, on=['country_code', 'TIME_PERIOD'], how='outer')
merged_monthly_data_df = merged_monthly_data_df.drop('PARTNER_Labels_y', axis=1)
merged_monthly_data_df = merged_monthly_data_df.rename(columns={'PARTNER_Labels_x': 'PARTNER_Labels', 'A': 'population_projections'})


In [12]:
df_macroeconomic = merged_monthly_data_df

In [13]:
df_macroeconomic

,PARTNER_Labels,country_code,TIME_PERIOD,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,Argentina,ARG,2005-01-01 00:00:00.000000000,1.987371e+11,4.352966e+11,39070501.0,11.510000,1.299209e+06,9.640000,1712.850000,89.050000,YR2005,39070501.0
1,Argentina,ARG,2005-02-01 00:00:00.000000000,2.015554e+11,4.382157e+11,39104363.5,11.390833,1.299391e+06,9.601667,1723.801667,89.016667,NaN,39104363.5
2,Argentina,ARG,2005-03-01 00:00:00.000000000,2.043738e+11,4.411348e+11,39138226.0,11.271667,1.299572e+06,9.563333,1734.753333,88.983333,NaN,39138226.0
3,Argentina,ARG,2005-04-01 00:00:00.000000000,2.071921e+11,4.440538e+11,39172088.5,11.152500,1.299754e+06,9.525000,1745.705000,88.950000,NaN,39172088.5
4,Argentina,ARG,2005-05-01 00:00:00.000000000,2.100105e+11,4.469729e+11,39205951.0,11.033333,1.299936e+06,9.486667,1756.656667,88.916667,NaN,39205951.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61747,NaN,ZWE,1970-01-01 00:00:00.000002019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2019,15354608.0
61748,NaN,ZWE,1970-01-01 00:00:00.000002020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2020,15669666.0
61749,NaN,ZWE,1970-01-01 00:00:00.000002021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2021,15993524.0
61750,NaN,ZWE,1970-01-01 00:00:00.000002022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2022,16320537.0


## MERGING DATA

### Start with empty dataframe with all possible combinations

In [15]:
import itertools
import pandas as pd

df_maize['TIME_PERIOD'] = pd.to_datetime(df_maize['TIME_PERIOD'])

# Get all unique values for TIME_PERIOD and PARTNER_Labels
time_periods = df_maize['TIME_PERIOD'].unique()
partner_labels = df_maize['PARTNER_Labels'].unique()

# Generate all possible combinations
combinations = list(itertools.product(time_periods, partner_labels))

# Create a new dataframe with the combinations
df_combinations = pd.DataFrame(combinations, columns=['TIME_PERIOD', 'PARTNER_Labels'])

# Merge with the original dataframe to get the corresponding values
df_result = pd.merge(df_combinations, df_maize, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')

# Fill any missing values with 0
df_result = df_result.fillna(0)
df_result.to_excel("eventesten.xlsx")

## Merge with global price data

In [17]:
df_result['TIME_PERIOD'] = pd.to_datetime(df_result['TIME_PERIOD'])
globalprice['TIME_PERIOD'] = pd.to_datetime(globalprice['TIME_PERIOD'])

# add globalprice as a new column to the dataframe
df_merged_result_globalprice = df_result.merge(globalprice, on=['TIME_PERIOD'], how='right')
df_merged_result_globalprice = df_merged_result_globalprice.rename(columns={"globalprice_y": "globalprice"})

df_merged_result_globalprice

,TIME_PERIOD,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice
0,2005-01-01,Afghanistan,0,0.0,0.00,0.000000,0.122419
1,2005-01-01,Antigua and Barbuda,0,0.0,0.00,0.000000,0.122419
2,2005-01-01,Argentina,maize,8100.0,937.50,0.115741,0.122419
3,2005-01-01,Australia,0,0.0,0.00,0.000000,0.122419
4,2005-01-01,Austria,0,0.0,0.00,0.000000,0.122419
...,...,...,...,...,...,...,...
23321,2022-10-01,United States,maize,560.6,248.89,0.443971,0.347175
23322,2022-10-01,Uruguay,0,0.0,0.00,0.000000,0.347175
23323,2022-10-01,Viet Nam,0,0.0,0.00,0.000000,0.347175
23324,2022-10-01,Zambia,0,0.0,0.00,0.000000,0.347175


### Merged trade data, oil price

In [18]:
df_merged_trade_oil = df_oil_price.merge(df_merged_result_globalprice, on='TIME_PERIOD', how='outer')
df_merged_trade_oil

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice
0,2005-01-01,42.972278,Afghanistan,0,0.0,0.00,0.000000,0.122419
1,2005-01-01,42.972278,Antigua and Barbuda,0,0.0,0.00,0.000000,0.122419
2,2005-01-01,42.972278,Argentina,maize,8100.0,937.50,0.115741,0.122419
3,2005-01-01,42.972278,Australia,0,0.0,0.00,0.000000,0.122419
4,2005-01-01,42.972278,Austria,0,0.0,0.00,0.000000,0.122419
...,...,...,...,...,...,...,...,...
23321,2022-10-01,90.326667,United States,maize,560.6,248.89,0.443971,0.347175
23322,2022-10-01,90.326667,Uruguay,0,0.0,0.00,0.000000,0.347175
23323,2022-10-01,90.326667,Viet Nam,0,0.0,0.00,0.000000,0.347175
23324,2022-10-01,90.326667,Zambia,0,0.0,0.00,0.000000,0.347175


### Merged trade data, oil price, weather data

In [19]:
df_merged_trade_oil_weather = df_merged_trade_oil.merge(df_global_temperature, on=['TIME_PERIOD', 'PARTNER_Labels'], how='outer')
df_merged_trade_oil_weather

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,2005-01-01,42.972278,Afghanistan,0,0.0,0.0,0.000000,0.122419,NaN,NaN,NaN
1,2005-01-01,42.972278,Antigua and Barbuda,0,0.0,0.0,0.000000,0.122419,NaN,NaN,NaN
2,2005-01-01,42.972278,Argentina,maize,8100.0,937.5,0.115741,0.122419,NaN,NaN,NaN
3,2005-01-01,42.972278,Australia,0,0.0,0.0,0.000000,0.122419,242.290980,162.67033,324.125000
4,2005-01-01,42.972278,Austria,0,0.0,0.0,0.000000,0.122419,-1.166667,-172.50000,125.000000
...,...,...,...,...,...,...,...,...,...,...,...
27666,2022-10-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,139.870035,25.80000,270.454545
27667,2022-11-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,69.985421,-21.00000,215.000000
27668,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,-32.452923,-173.70000,113.750000
27669,2022-11-01,NaN,Viet Nam,NaN,NaN,NaN,NaN,NaN,272.000000,252.00000,304.000000


### Merged trade data, oil price, weather, future prices data

In [23]:
df_merged_trade_oil_weather_futures = df_merged_trade_oil_weather.merge(df_futures_corn, on='TIME_PERIOD', how='outer')
df_merged_trade_oil_weather_futures.head(2500)

df_merged_trade_oil_weather_futures.to_excel("eventesten2.xlsx")
df_merged_trade_oil_weather_futures

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,MAX_TMAX,Corn Price Futures
0,2005-01-01,42.972278,Afghanistan,0,0.0,0.0,0.000000,0.122419,NaN,NaN,NaN,204.5
1,2005-01-01,42.972278,Antigua and Barbuda,0,0.0,0.0,0.000000,0.122419,NaN,NaN,NaN,204.5
2,2005-01-01,42.972278,Argentina,maize,8100.0,937.5,0.115741,0.122419,NaN,NaN,NaN,204.5
3,2005-01-01,42.972278,Australia,0,0.0,0.0,0.000000,0.122419,242.290980,162.670330,324.125000,204.5
4,2005-01-01,42.972278,Austria,0,0.0,0.0,0.000000,0.122419,-1.166667,-172.500000,125.000000,204.5
...,...,...,...,...,...,...,...,...,...,...,...,...
27666,2022-12-01,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,33.276307,-78.000000,163.333333,NaN
27667,2022-12-01,NaN,United States,NaN,NaN,NaN,NaN,NaN,-14.536099,-125.184513,64.457218,NaN
27668,2022-12-01,NaN,Burkina Faso,NaN,NaN,NaN,NaN,NaN,260.295430,160.333333,373.500000,NaN
27669,2022-12-01,NaN,Uzbekistan,NaN,NaN,NaN,NaN,NaN,-32.452923,-173.700000,113.750000,NaN


### Merged trade, oil price, weather, future prices, macroeconomic 

In [25]:
df_merged = df_merged_trade_oil_weather_futures.merge(df_macroeconomic, on=['TIME_PERIOD', 'PARTNER_Labels'], how='outer')
df_merged.to_excel("boer.xlsx", index=True)



,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01 00:00:00.000000000,42.972278,Afghanistan,0,0.0,0.0,0.000000,0.122419,NaN,NaN,...,6.226199e+09,9.275174e+09,24411191.0,NaN,379100.0,33.88,NaN,NaN,YR2005,2.441119e+07
1,2005-01-01 00:00:00.000000000,42.972278,Antigua and Barbuda,0,0.0,0.0,0.000000,0.122419,NaN,NaN,...,1.022963e+09,1.231883e+09,79869.0,NaN,90.0,0.00,1715.31,0.00,YR2005,7.986900e+04
2,2005-01-01 00:00:00.000000000,42.972278,Argentina,maize,8100.0,937.5,0.115741,0.122419,NaN,NaN,...,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,3.907050e+07
3,2005-01-01 00:00:00.000000000,42.972278,Australia,0,0.0,0.0,0.000000,0.122419,242.290980,162.67033,...,6.953285e+11,1.027253e+12,20176844.0,5.03,4102300.0,6.71,5624.19,94.22,YR2005,2.017684e+07
4,2005-01-01 00:00:00.000000000,42.972278,Austria,0,0.0,0.0,0.000000,0.122419,-1.166667,-172.50000,...,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8.227829e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66075,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,4.367088e+06
66076,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,7.491934e+09
66077,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,2.927400e+07
66078,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,1.676776e+07


## CLEANING EXTREME VALUES

In [26]:
import numpy as np

# Delete all inf and -inf
df_merged = df_merged.replace([np.inf, -np.inf], np.nan)

# Replace all zeros with NaN
df_merged = df_merged.replace(0, np.nan)

df_merged

,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01 00:00:00.000000000,42.972278,Afghanistan,NaN,NaN,NaN,NaN,0.122419,NaN,NaN,...,6.226199e+09,9.275174e+09,24411191.0,NaN,379100.0,33.88,NaN,NaN,YR2005,2.441119e+07
1,2005-01-01 00:00:00.000000000,42.972278,Antigua and Barbuda,NaN,NaN,NaN,NaN,0.122419,NaN,NaN,...,1.022963e+09,1.231883e+09,79869.0,NaN,90.0,NaN,1715.31,NaN,YR2005,7.986900e+04
2,2005-01-01 00:00:00.000000000,42.972278,Argentina,maize,8100.0,937.5,0.115741,0.122419,NaN,NaN,...,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,3.907050e+07
3,2005-01-01 00:00:00.000000000,42.972278,Australia,NaN,NaN,NaN,NaN,0.122419,242.290980,162.67033,...,6.953285e+11,1.027253e+12,20176844.0,5.03,4102300.0,6.71,5624.19,94.22,YR2005,2.017684e+07
4,2005-01-01 00:00:00.000000000,42.972278,Austria,NaN,NaN,NaN,NaN,0.122419,-1.166667,-172.50000,...,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8.227829e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66075,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,4.367088e+06
66076,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,7.491934e+09
66077,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,2.927400e+07
66078,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,1.676776e+07


In [27]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'gdp_current_us', 'population_total', 'unemployment_total', 'renewable_energy_consumption_perc_of_total', 'energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)

df_merged


,TIME_PERIOD,OIL_PRICE $/bbl,PARTNER_Labels,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,globalprice,AVG_TAVG,MIN_TMIN,...,gdp_current_us,gdp_constant_2015_us,population_total,unemployment_total,agricultural_land,renewable_energy_consumption_perc_of_total,energy_use_kg_of_oil_equivalent_per_capita,fossil_fuel_energy_consumption_perc_of_total,Time Code,population_projections
0,2005-01-01 00:00:00.000000000,42.972278,Afghanistan,NaN,NaN,NaN,NaN,0.122419,NaN,NaN,...,6.226199e+09,9.275174e+09,24411191.0,NaN,379100.0,33.88,NaN,NaN,YR2005,24411191.0
1,2005-01-01 00:00:00.000000000,42.972278,Antigua and Barbuda,NaN,NaN,NaN,NaN,0.122419,NaN,NaN,...,1.022963e+09,1.231883e+09,79869.0,NaN,90.0,NaN,1715.31,NaN,YR2005,79869.0
2,2005-01-01 00:00:00.000000000,42.972278,Argentina,maize,8100.0,937.5,0.115741,0.122419,NaN,NaN,...,1.987371e+11,4.352966e+11,39070501.0,11.51,1299209.0,9.64,1712.85,89.05,YR2005,39070501.0
3,2005-01-01 00:00:00.000000000,42.972278,Australia,NaN,NaN,NaN,NaN,0.122419,242.290980,162.67033,...,6.953285e+11,1.027253e+12,20176844.0,5.03,4102300.0,6.71,5624.19,94.22,YR2005,20176844.0
4,2005-01-01 00:00:00.000000000,42.972278,Austria,NaN,NaN,NaN,NaN,0.122419,-1.166667,-172.50000,...,3.160923e+11,3.396552e+11,8227829.0,5.68,28568.0,24.41,4085.57,77.03,YR2005,8227829.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66075,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,4367088.0
66076,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,NaN
66077,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,29274002.0
66078,1970-01-01 00:00:00.000002016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YR2016,16767761.0


## FULL DATAFRAME (without filtering top countries)

In [67]:
pd.set_option('display.max_columns', 2000) #replace n with the number of columns you want to see completely
###FOR NOW
df_maize_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['globalprice', 'OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections'], margins=False)
df_maize_total['price']


PARTNER_Labels,Antigua and Barbuda,Argentina,Australia,Austria,Bangladesh,Belgium,Bosnia and Herzegovina,Brazil,Bulgaria,Burundi,Cameroon,Canada,Chile,China,Colombia,Costa Rica,Croatia,Cyprus,Czechia,Côte d'Ivoire,Denmark,Dominican Republic,Ecuador,Egypt,Estonia,Eswatini,Ethiopia,Finland,France,Gambia,Germany,Ghana,Greece,Guatemala,Hungary,India,Indonesia,Ireland,Israel,Italy,Japan,Jordan,Kazakhstan,Kenya,"Korea, Republic of",Lao Peoples Democratic Republic,Latvia,Lebanon,Libya,Lithuania,Luxembourg,Malaysia,Mali,Malta,Mexico,Moldova,Montenegro,Morocco,Mozambique,Myanmar,New Zealand,Nigeria,Norway,Pakistan,Paraguay,Peru,Poland,Portugal,Qatar,Romania,Russia,Saudi Arabia,Serbia,Slovakia,Slovenia,Somalia,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Syrian Arab Republic,Taiwan,Tanzania,Thailand,Togo,Trinidad and Tobago,Türkiye,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Viet Nam,Zambia,Zimbabwe
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-01,NaN,0.115741,NaN,NaN,NaN,0.119103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.122632,NaN,0.120429,NaN,NaN,NaN,0.109604,3.300000,NaN,NaN,NaN,0.037968,NaN,NaN,NaN,2.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.975000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.469281,NaN,NaN,0.710000,NaN,NaN,NaN,0.314011,0.334963,NaN,NaN,NaN,3.080
2005-02-01,NaN,0.123755,NaN,NaN,NaN,0.118607,NaN,0.164250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.124028,NaN,0.109035,NaN,NaN,NaN,0.105578,3.054545,NaN,NaN,NaN,0.295709,NaN,NaN,NaN,2.620000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.847059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.377869,NaN,NaN,NaN,NaN,NaN,NaN,0.380808,0.270379,NaN,NaN,NaN,2.550
2005-03-01,NaN,0.120312,NaN,0.133137,NaN,0.121043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.25,NaN,NaN,NaN,NaN,0.123698,NaN,0.118774,NaN,NaN,NaN,0.102434,3.425000,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.118764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.148000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.700000,NaN,NaN,NaN,NaN,NaN,NaN,3.437500,NaN,NaN,NaN,NaN,NaN,NaN,0.329100,0.274034,NaN,NaN,NaN,1.900
2005-04-01,NaN,0.109644,NaN,NaN,NaN,0.122473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117439,NaN,0.109957,NaN,NaN,NaN,0.104302,3.088889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,3.427273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.478587,NaN,NaN,NaN,NaN,NaN,NaN,0.292759,0.365034,NaN,NaN,NaN,0.775
2005-05-01,NaN,0.130592,NaN,NaN,NaN,0.129335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.123325,NaN,0.116828,NaN,NaN,NaN,0.115436,3.080000,NaN,NaN,NaN,0.500000,NaN,NaN,NaN,2.045455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.971429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.675000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.363068,NaN,NaN,NaN,NaN,NaN,NaN,0.336029,0.298552,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,NaN,0.429998,NaN,0.540927,NaN,0.426859,NaN,0.211752,0.540741,NaN,NaN,0.550000,NaN,0.431646,NaN,NaN,0.544000,NaN,0.215304,NaN,0.540331,NaN,0.45,NaN,0.539130,NaN,NaN,0.547500,0.289791,NaN,0.340837,NaN,0.540418,NaN,0.434541,NaN

## HANDLE MISSING DATA

### Fill in missing prices with trend in France, Germany WE DO NOT USE THIS ANYMORE.... BUT STILL FOR REFERENCE

In [ ]:
""""
# FOR NOW
df_maize_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections'], margins=True)

# Create a new DataFrame that only contains the 'price' values of France and Germany:
df_price = df_maize_total[['price']].loc[:, ('price', ['France', 'Germany'])]

# Remove any rows that contain missing values (NaN) in the 'price' column for France and Germany
df_price = df_price.dropna()

df_price_mean = df_price.mean(axis=1)

# Calculate the year-over-year percent change in 'price' for France and Germany:
df_price_pct_change = df_price.pct_change()

# Replace NaN values with 0 (there are NaN values for the first year)
df_price_pct_change = df_price_pct_change.fillna(0)

# Calculate the mean percent change in 'price' for each year across France and Germany:
df_price_pct_change_mean = df_price_pct_change.mean(axis=1)

# Create a new DataFrame that only contains the missing 'price' values for other countries:
df_missing_price = df_maize_total[['price']].loc[:, ('price', df_maize_total['price'].isna().any())]

# Fill in the missing 'price' values for each year using the mean percent change from France and Germany
df_missing_price['price'] = df_missing_price['price'].fillna(method='ffill')
df_missing_price['price'] = df_missing_price['price'].fillna(method='bfill')
df_missing_price['price'] = df_missing_price['price'].fillna(df_price.iloc[-1]['price'])
df_missing_price['price'] = df_missing_price['price'].apply(lambda x: x * (1 + df_price_pct_change_mean))

# Update NaN values in df_maize_total with non-null values from df_missing_price
mask = df_maize_total['price'].isna() # create a mask of NaN values in the price column
df_maize_total[mask] = df_missing_price[mask] # update only the NaN values using the mask

# Update the original DataFrame with the filled-in missing values
df_maize_total_updated = df_maize_total.combine_first(df_missing_price)

df_missing_price
"""

## Fill in missing values with global price

In [68]:
# Create a new DataFrame that only contains the missing 'price' values for other countries:
df_missing_price = df_maize_total[['price']].loc[:, ('price', df_maize_total['price'].isna().any())]
df_missing_price = df_missing_price.iloc[:-2]
# Forward fill any remaining missing values
df_missing_price['price'] = df_missing_price['price'].fillna(method='ffill')
# Back fill any missing values at the beginning of the series
df_missing_price['price'] = df_missing_price['price'].fillna(method='bfill')


In [69]:
###FOR NOW
#df_maize_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections'], margins=True)

# Create a new DataFrame that only contains the globalprice
df_price = globalprice['globalprice']
df_price = pd.DataFrame(df_price)
df_price.set_index(globalprice['TIME_PERIOD'], inplace=True)

# Calculate the year-over-year percent change in 'price' for France and Germany:
df_price_pct_change = df_price.pct_change()

# Replace NaN values with 0 (there are NaN values for the first year)
df_price_pct_change = df_price_pct_change.fillna(0)

df_price_pct_change = pd.DataFrame(df_price_pct_change)

In [70]:
multiplyfactor = 1 + df_price_pct_change['globalprice']
multiplyfactor = pd.DataFrame(multiplyfactor)

In [71]:
df_result = df_missing_price.multiply(multiplyfactor['globalprice'], axis='index')

In [72]:
mask = df_maize_total['price'].isna()

In [73]:
df_maize_total_updated = df_maize_total.combine_first(df_result[mask])
df_maize_total_updated['price']

PARTNER_Labels,Antigua and Barbuda,Argentina,Australia,Austria,Bangladesh,Belgium,Bosnia and Herzegovina,Brazil,Bulgaria,Burundi,Cameroon,Canada,Chile,China,Colombia,Costa Rica,Croatia,Cyprus,Czechia,Côte d'Ivoire,Denmark,Dominican Republic,Ecuador,Egypt,Estonia,Eswatini,Ethiopia,Finland,France,Gambia,Germany,Ghana,Greece,Guatemala,Hungary,India,Indonesia,Ireland,Israel,Italy,Japan,Jordan,Kazakhstan,Kenya,"Korea, Republic of",Lao Peoples Democratic Republic,Latvia,Lebanon,Libya,Lithuania,Luxembourg,Malaysia,Mali,Malta,Mexico,Moldova,Montenegro,Morocco,Mozambique,Myanmar,New Zealand,Nigeria,Norway,Pakistan,Paraguay,Peru,Poland,Portugal,Qatar,Romania,Russia,Saudi Arabia,Serbia,Slovakia,Slovenia,Somalia,South Africa,Spain,Sri Lanka,Sweden,Switzerland,Syrian Arab Republic,Taiwan,Tanzania,Thailand,Togo,Trinidad and Tobago,Türkiye,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Viet Nam,Zambia,Zimbabwe
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-01,0.392593,0.115741,0.157143,0.133137,0.175000,0.119103,0.400000,0.164250,0.224834,0.209091,3.400000,0.417000,1.662393,0.200000,0.400000,0.466667,4.500000,2.800000,0.126499,1.500000,4.350000,0.300000,0.350000,4.250000,3.600000,3.200000,1.700000,0.500000,0.122632,0.300000,0.120429,0.300000,0.870000,3.650000,0.109604,3.300000,1.800000,0.426190,3.000000,0.037968,1.500000,0.800000,0.268031,2.583333,1.300000,4.200000,5.000000,0.500000,0.500000,0.350000,0.118764,3.971429,0.450000,0.700000,0.150000,0.203884,1.900000,0.250000,0.166667,0.400000,3.504348,0.900000,1.370652,0.200000,0.142805,3.975000,0.118250,0.690400,2.000000,0.209979,0.277439,1.085714,0.138992,0.095001,0.850000,0.079999,4.100000,1.700000,1.000000,0.566667,2.000000,0.725000,1.900000,4.000000,3.469281,2.469231,0.425000,0.710000,1.200000,0.203184,0.242762,0.314011,0.334963,0.301366,3.645455,4.847368,3.080000
2005-02-01,0.380277,0.123755,0.152213,0.128961,0.169510,0.118607,0.387452,0.164250,0.217780,0.202532,3.293339,0.403918,1.610242,0.193726,0.387452,0.452027,4.358831,2.712161,0.122531,1.452944,4.213536,0.290589,0.339020,4.116673,3.487064,3.099613,1.646669,0.484315,0.124028,0.290589,0.109035,0.290589,0.842707,3.535496,0.105578,3.054545,1.743532,0.412820,2.905887,0.295709,1.452944,0.774903,0.259622,2.620000,1.259218,4.068242,4.843145,0.484315,0.484315,0.339020,0.115039,3.846841,0.435883,0.678040,0.145294,0.197488,1.840395,0.242157,0.161438,0.387452,3.394413,0.871766,1.327653,0.193726,0.138325,3.847059,0.114540,0.668741,1.937258,0.203392,0.268735,1.051654,0.134632,0.092021,0.823335,0.077490,3.971379,1.646669,0.968629,0.548890,1.937258,0.702256,1.840395,3.874516,3.377869,2.391769,0.411667,0.687727,1.162355,0.196810,0.235146,0.380808,0.270379,0.291912,3.531093,4.695302,2.550000
2005-03-01,0.401925,0.120312,0.160878,0.133137,0.179160,0.121043,0.409508,0.168154,0.230178,0.214061,3.480818,0.426912,1.701908,0.204754,0.409508,0.477759,4.606965,2.866556,0.129506,1.535655,4.453400,0.307131,0.358320,4.250000,3.685572,3.276064,1.740409,0.511885,0.123698,0.307131,0.118774,0.307131,0.890680,3.736761,0.102434,3.425000,1.842786,0.436321,3.000000,0.302738,1.535655,0.819016,0.274402,2.682278,1.330901,4.299834,5.118850,0.511885,0.511885,0.358320,0.118764,4.065830,0.460697,0.716639,0.153566,0.208730,1.945163,0.255943,0.170628,0.409508,3.587646,0.921393,1.403233,0.204754,0.146200,3.148000,0.121061,0.706811,2.047540,0.214970,0.284033,1.111522,0.142296,0.097259,0.870205,0.081901,4.197457,1.700000,1.023770,0.580136,2.047540,0.742233,1.945163,4.095080,3.437500,2.527925,0.435102,0.726877,1.228524,0.208014,0.248533,0.329100,0.274034,0.308529,3.732107,4.962591,1.900000
2005-04-01,0.375365,0.109644,0.150247,0.127295,0.167321,0.122473,0.382447,0.157042,0.214967,0.199915,3.250799,0.398701,1.589443,0.191223,0.382447,0.446188,4.302528,2.677129,0.120948,1.434176,4.350000,0.286835,0.334641,4.063499,3.442023,3.059576,1.625400,0.478059,0.117439,0.286835,0.109957,0.286835,0.831

### Interpolate macroeconomic

In [74]:
# fill missing values at the beginning of each column with the first known value
df_maize_total_updated.fillna(method='ffill')

# fill missing values at the end of each column with the last known value
df_maize_total_updated.fillna(method='bfill')

# interpolate missing values for selected columns
cols_to_interpolate = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']

for col in cols_to_interpolate:
    df_maize_total_updated.loc[:, (col, slice(None))] = df_maize_total.loc[:, (col, slice(None))].interpolate()
    df_maize_total_updated.fillna(method='bfill')
    df_maize_total_updated.fillna(method='ffill')

# print the updated pivot table dataframe
df_maize_total_updated


AVG_TAVG                                             \
PARTNER_Labels     Albania     Algeria Angola     Armenia   Australia   
TIME_PERIOD                                                             
2005-01-01             NaN   76.228202  274.0  -87.478261  242.290980   
2005-02-01             NaN   84.478184  270.5  -87.000000  237.755531   
2005-03-01             NaN  147.382230  156.0  -46.440000  223.480383   
2005-04-01             NaN  181.541272  242.5   39.714286  208.498275   
2005-05-01             NaN  240.048870  211.5   96.043478  167.861508   
...                    ...         ...    ...         ...         ...   
2022-08-01      144.666667  312.218029  213.0  209.490002  146.200527   
2022-09-01      144.666667  286.008954  213.0  173.108906  165.721311   
2022-10-01      144.666667  231.950455  213.0  111.054056  189.637400   
2022-11-01      144.666667  168.537960  213.0   50.784508  201.174639   
2022-12-01      144.666667  150.914659  213.0   -1.926037  224.700188   

                                                                            \
PARTNER_Labels     Austria  Azerbaijan     Bahrain     Belarus     Belgium   
TIME_PERIOD                                                                  
2005-01-01       -1.166667         NaN  160.222222   -7.788025   30.700000   
2005-02-01      -42.166667   72.000000  176.000000  -59.820327   20.000000   
2005-03-01       -1.994624   68.000000  211.095238  -32.497550   55.800000   
2005-04-01       60.233333  191.000000  257.315789   80.398109  111.000000   
2005-05-01      108.758065  200.000000  303.117647  130.261818  129.250000   
...                    ...         ...         ...         ...         ...   
2022-08-01      175.258849  260.574138  360.153846  207.011045  213.200000   
2022-09-01      103.779082  218.628333  346.350000   99.815254  166.000000   
2022-10-01       98.433871  160.495556  307.000000   82.525416  147.333333   
2022-11-01       36.939583   96.659524  274.357143   17.335752   88.454545   
2022-12-01       -9.296552   37.345238  223.176471  -29.105331   36.900000   

                                                                            \
PARTNER_Labels Bosnia and Herzegovina      Brazil    Bulgaria Burkina Faso   
TIME_PERIOD                                                                  
2005-01-01                 -14.969892  269.701032   24.086022   253.052381   
2005-02-01                 -36.017857  266.743474   -9.425595   302.660095   
2005-03-01                  20.650538  266.528149   54.857527   332.319386   
2005-04-01                  78.601667  253.719444  113.894444   336.259203   
2005-05-01                 134.995161  247.859946  170.809140   315.863580   
...                               ...         ...         ...          ...   
2022-08-01                 218.941935  222.129167  243.446237   260.923571   
2022-09-01                 189.631034  221.270833  180.200000   260.132837   
2022-10-01                 172.066092  242.066667  138.333333   280.302304   
2022-11-01                 117.805769  238.194444   96.594444   271.984864   
2022-12-01                  88.849383  249.907407   49.658602   260.295430   

                                                                            \
PARTNER_Labels    Cameroon      Canada       Chile       China    Colombia   
TIME_PERIOD                                                                  
2005-01-01      265.058824         NaN  193.505177  -39.628008  227.893553   
2005-02-01      301.824074         NaN  194.394702  -25.939147  254.429167   
2005-03-01      328.450000 -104.471986  179.134537   50.029574  258.143750   
2005-04-01      310.826087  -20.064078  152.958690  132.430808  251.740190   
2005-05-01      290.272727   39.783135  138.762942  176.286936  251.447116   
...                    ...         ...         ...         ...         ...   
2022-08-01      231.275000  179.322868   98.776587  234.647872  248.137771   
2022-09-01      224.533333  1

## SAVE EXCEL FILE FULL DATAFRAME

In [75]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'MAIZE_FULL_{now}.xlsx'
df_maize_total_updated.to_excel(filename, index=True)

## STANDARDIZATION

In [76]:
from sklearn.preprocessing import StandardScaler

df_maize_total_updated_std = df_maize_total_updated.copy()

# get a list of all the column names in the DataFrame
all_cols = list(df_maize_total_updated_std.columns)

# create a list of all columns to normalize, excluding 'price' for all countries
cols_to_normalize = [col for col in all_cols if 'price' not in col]

# instantiate the StandardScaler object
scaler = StandardScaler()

# fit the scaler on the selected columns
scaler.fit(df_maize_total_updated_std[cols_to_normalize])

# transform the selected columns using the scaler
df_maize_total_updated_std[cols_to_normalize] = scaler.transform(df_maize_total_updated_std[cols_to_normalize])
df_maize_total_updated_std

C:\Users\basbe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\basbe\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


AVG_TAVG                                                    \
PARTNER_Labels   Albania   Algeria    Angola   Armenia Australia   Austria   
TIME_PERIOD                                                                  
2005-01-01           NaN -1.625629  1.034987 -1.798018  1.252027 -1.083560   
2005-02-01           NaN -1.515783  0.895962 -1.792527  1.137852 -1.677014   
2005-03-01           NaN -0.678237 -3.652156 -1.326795  0.778492 -1.095545   
2005-04-01           NaN -0.223420 -0.216241 -0.337526  0.401334 -0.194828   
2005-05-01           NaN  0.555589 -1.447610  0.309276 -0.621650  0.507542   
...                  ...       ...       ...       ...       ...       ...   
2022-08-01     -0.353314  1.516497 -1.388027  1.611930 -1.166940  1.470106   
2022-09-01     -0.353314  1.167532 -1.388027  1.194183 -0.675527  0.435474   
2022-10-01     -0.353314  0.447761 -1.388027  0.481636 -0.073467  0.358105   
2022-11-01     -0.353314 -0.396555 -1.388027 -0.210411  0.216970 -0.531992   
2022-12-01     -0.353314 -0.631203 -1.388027 -0.815662  0.809199 -1.201236   

                                                         \
PARTNER_Labels Azerbaijan   Bahrain   Belarus   Belgium   
TIME_PERIOD                                               
2005-01-01            NaN -1.872269 -0.977534 -1.407486   
2005-02-01      -1.056052 -1.628760 -1.582130 -1.595379   
2005-03-01      -1.105865 -1.087113 -1.264649 -0.966726   
2005-04-01       0.425891 -0.373761  0.047158  0.002594   
2005-05-01       0.537971  0.333129  0.626557  0.323066   
...                   ...       ...       ...       ...   
2022-08-01       1.292319  1.213405  1.518355  1.797240   
2022-09-01       0.769955  1.000361  0.272778  0.968402   
2022-10-01       0.046010  0.393048  0.071877  0.640612   
2022-11-01      -0.748960 -0.110751 -0.685604 -0.393308   
2022-12-01      -1.487619 -0.900655 -1.225233 -1.298613   

                                                                        \
PARTNER_Labels Bosnia and Herzegovina    Brazil  Bulgaria Burkina Faso   
TIME_PERIOD                                                              
2005-01-01                  -1.828173  1.315208 -1.272341    -1.362267   
2005-02-01                  -2.108647  1.191699 -1.691326     0.632883   
2005-03-01                  -1.353515  1.182707 -0.887616     1.825736   
2005-04-01                  -0.581289  0.647810 -0.149497     1.984190   
2005-05-01                   0.170181  0.403115  0.562088     1.163908   
...                               ...       ...       ...          ...   
2022-08-01                   1.288810 -0.671414  1.470245    -1.045700   
2022-09-01                   0.898229 -0.707258  0.679499    -1.077502   
2022-10-01                   0.664168  0.161185  0.156054    -0.266315   
2022-11-01                  -0.058876 -0.000521 -0.365793    -0.600830   
2022-12-01                  -0.444733  0.488618 -0.952616    -1.070962   

                                                                  \
PARTNER_Labels  Cameroon    Canada     Chile     China  Colombia   
TIME_PERIOD                                                        
2005-01-01     -0.036075       NaN  1.463038 -1.586006 -0.527733   
2005-02-01      1.411027       NaN  1.489391 -1.443774  0.675055   
2005-03-01      2.459040 -1.295559  1.037288 -0.654437  0.843427   
2005-04-01      1.765352 -0.429196  0.261793  0.201736  0.553170   
2005-05-01      0.956359  0.185077 -0.158775  0.657414  0.539886   
...                  ...       ...       ...       ...       ...   
2022-08-01     -1.365827  1.617314 -1.343425  1.263801  0.389883   
2022-09-01     -1.631183  1.122685 -0.648089  0.844932  0.404835   
2022-10-01     -1.720400  0.521893  0.205445  0.062542 -0.086596   
2022-11-01     -1.809618 -0.600848  0.499614 -0.534900 -0.060458   
2022-12-01     -1.809618 -1.290403  1.053763 -1.512325  0.031090   

                                                                              \
PARTNER_Labels Congo,  Dem

## SAVE EXCEL FILE

### STANDARDIZED FULL DATA

In [77]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'MAIZE_FULL_STANDARDIZED_{now}.xlsx'
df_maize_total_updated_std.to_excel(filename, index=True)

## ANALYSIS CORN IMPORT TO NL (HISTORICAL) & FILTER TOP PARTNERS

In [1]:
df_maize_corn_0['TIME_PERIOD'] = pd.to_datetime(df_maize_corn_0['TIME_PERIOD'])
mask = (df_maize_corn_0['TIME_PERIOD'] >= '2005-01-01') & (df_maize_corn_0['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_maize_corn_0.loc[mask]
total = df_filtered['OBS_VALUE_QUANTITY_TON'].sum()

df_maize_corn_0

NameError: name 'pd' is not defined

In [79]:
df_maize_corn_0['TIME_PERIOD'] = pd.to_datetime(df_maize_corn_0['TIME_PERIOD'])
mask = (df_maize_corn_0['TIME_PERIOD'] >= '2005-01-01') & (df_maize_corn_0['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_maize_corn_0.loc[mask]

grouped = df_filtered.groupby('PARTNER_Labels')['OBS_VALUE_QUANTITY_TON'].sum().reset_index()
grouped = grouped.sort_values(by='OBS_VALUE_QUANTITY_TON', ascending=False)
grouped = grouped.reset_index()
grouped['cumulative_sum'] = grouped['OBS_VALUE_QUANTITY_TON'].cumsum()
grouped['cumulative_sum_pct'] = grouped['cumulative_sum'] / grouped['OBS_VALUE_QUANTITY_TON'].sum()

grouped.head(10)        
        


,index,PARTNER_Labels,OBS_VALUE_QUANTITY_TON,cumulative_sum,cumulative_sum_pct
0,101,Ukraine,22670363.9,22670363.9,0.293292
1,32,France,20818695.8,43489059.7,0.562628
2,9,Brazil,7514296.3,51003356.0,0.659842
3,35,Germany,7132645.5,58136001.5,0.752119
4,40,Hungary,3995159.3,62131160.8,0.803805
5,79,Romania,3816059.5,65947220.3,0.853174
6,6,Belgium,3008019.1,68955239.4,0.892090
7,80,Russia,1464272.2,70419511.6,0.911033
8,2,Argentina,1357260.3,71776771.9,0.928593
9,104,United States,1045888.5,72822660.4,0.942124


In [80]:
# Declare splitting point for cumulative import to NL history, biggest partners
splitting_point = 0.8

for i, row in grouped.iterrows():
    if row['cumulative_sum_pct'] >= splitting_point:
        top_x = i + 1
        break
        
top_partners = grouped['PARTNER_Labels'].head(top_x).tolist()
top_partners

['Ukraine', 'France', 'Brazil', 'Germany', 'Hungary']

## FILTERING TOP PARTNERS

In [81]:
pd.set_option('display.max_columns', 90) #replace n with the number of columns you want to see completely

### FULL DATA

In [82]:
# filter the pivot table using the loc method
df_maize_total_filtered = df_maize_total_updated.loc[:, (slice(None), top_partners)]


# GEEEN IDEE WAAROM, MAAR NA FILTEREN VERANDERT PIVOT OPEENS WEER NAAR VOLGORDE DIE WE EERST VOOR OGEN HADDEN, MWEH
# display the filtered pivot table
df_maize_total_filtered = df_maize_total_filtered.sort_index(axis=1)
df_maize_total_filtered.head(5)

AVG_TAVG                                                  \
PARTNER_Labels      Brazil      France     Germany     Hungary     Ukraine   
TIME_PERIOD                                                                  
2005-01-01      269.701032   50.081198   28.651592   -1.306452    0.869290   
2005-02-01      266.743474   30.689158   -1.962406  -25.371429  -38.376438   
2005-03-01      266.528149   76.202630   45.730051   38.383871   -3.794989   
2005-04-01      253.719444  108.081105  101.859649  112.460000   90.622214   
2005-05-01      247.859946  147.068032  134.667233  162.993548  161.493475   

               Corn Price Futures                                 MAX_TMAX  \
PARTNER_Labels             Brazil  France Germany Hungary Ukraine   Brazil   
TIME_PERIOD                                                                  
2005-01-01                 204.50  204.50  204.50  204.50  204.50   355.50   
2005-02-01                 222.75  222.75  222.75  222.75  222.75   342.50   
2005-03-01                 221.00  221.00  221.00  221.00  221.00   347.25   
2005-04-01                 213.50  213.50  213.50  213.50  213.50   338.50   
2005-05-01                 230.75  230.75  230.75  230.75  230.75   341.00   

                                                               MIN_TMIN  \
PARTNER_Labels      France     Germany     Hungary     Ukraine   Brazil   
TIME_PERIOD                                                               
2005-01-01      124.038462  129.842105  137.000000  102.142857   182.00   
2005-02-01      119.983051  108.315789   57.666667   60.142857   180.00   
2005-03-01      201.140625  180.894737  209.000000  112.468208   201.75   
2005-04-01      254.950820  213.315789  218.000000  204.189944   150.00   
2005-05-01      279.101695  314.684211  321.000000  307.400000   138.00   

                                                               \
PARTNER_Labels     France     Germany     Hungary     Ukraine   
TIME_PERIOD                                                     
2005-01-01     -40.377358  -91.000000 -136.000000 -105.930233   
2005-02-01     -65.083333 -125.684211 -167.600000 -190.275862   
2005-03-01     -90.450000  -99.526316 -126.400000 -123.523810   
2005-04-01      23.620690  -12.421053   -2.000000  -37.809524   
2005-05-01      56.158730   22.631579   27.833333   57.571429   

               OBS_VALUE_QUANTITY_TON                                      \
PARTNER_Labels                 Brazil    France  Germany  Hungary Ukraine   
TIME_PERIOD                                                                 
2005-01-01                        NaN  106983.9  39590.1   4705.2     NaN   
2005-02-01                  40.000000  126648.5  73452.7  16053.2     NaN   
2005-03-01                  40.230769   94526.9  66027.6  14437.1     NaN   
2005-04-01                  40.461538  137965.3  52878.4   6912.4     NaN   
2005-05-01                  40.692308  122365.8  42554.1  19030.3     NaN   

               OBS_VALUE_VALUE_1000EURO                                        \
PARTNER_Labels                   Brazil    France  Germany  Hungary   Ukraine   
TIME_PERIOD                                                                     
2005-01-01                          NaN  13119.68  4767.79   515.71       NaN   
2005-02-01                     6.570000  15708.00  8008.91  1694.86       NaN   
2005-03-01                     7.343077  11692.80  7842.34  1478.85       NaN   
2005-04-01                     8.116154  16202.52  5814.35   720.98  0.080000   
2005-05-01                     8.889231  15090.74  4971.52  2196.78  0.085714   

               OIL_PRICE $/bbl                                              \
PARTNER_Labels          Brazil     France    Germany    Hungary    Ukraine   
TIME_PERIOD                                                                  
2005-01-01           42.972278  42.972278  42.972278  42.972278  42.972278   
2005-02-01           44.818211  44.818211  44.818211  44.818211  44.

#### SAVE EXCEL FILE FILTERED NON STANDARDIZED

In [83]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'MAIZE_FILTERED_{now}.xlsx'
df_maize_total_filtered.to_excel(filename, index=True)

### STANDARDIZED DATA

In [84]:
# filter the pivot table using the loc method
df_maize_total_filtered_std = df_maize_total_updated_std.loc[:, (slice(None), top_partners)]


# GEEEN IDEE WAAROM, MAAR NA FILTEREN VERANDERT PIVOT OPEENS WEER NAAR VOLGORDE DIE WE EERST VOOR OGEN HADDEN, MWEH
# display the filtered pivot table
df_maize_total_filtered_std = df_maize_total_filtered_std.sort_index(axis=1)
df_maize_total_filtered_std.head(5)

AVG_TAVG                                          \
PARTNER_Labels    Brazil    France   Germany   Hungary   Ukraine   
TIME_PERIOD                                                        
2005-01-01      1.315208 -1.358269 -1.169212 -1.484828 -1.061958   
2005-02-01      1.191699 -1.709310 -1.641851 -1.789502 -1.499566   
2005-03-01      1.182707 -0.885410 -0.905543 -0.982329 -1.113967   
2005-04-01      0.647810 -0.308336 -0.038976 -0.044488 -0.061172   
2005-05-01      0.403115  0.397419  0.467529  0.595291  0.729075   

               Corn Price Futures                                          \
PARTNER_Labels             Brazil    France   Germany   Hungary   Ukraine   
TIME_PERIOD                                                                 
2005-01-01              -1.619096 -1.619096 -1.619096 -1.619096 -1.619096   
2005-02-01              -1.498452 -1.498452 -1.498452 -1.498452 -1.498452   
2005-03-01              -1.510020 -1.510020 -1.510020 -1.510020 -1.510020   
2005-04-01              -1.559600 -1.559600 -1.559600 -1.559600 -1.559600   
2005-05-01              -1.445567 -1.445567 -1.445567 -1.445567 -1.445567   

                MAX_TMAX                                          MIN_TMIN  \
PARTNER_Labels    Brazil    France   Germany   Hungary   Ukraine    Brazil   
TIME_PERIOD                                                                  
2005-01-01      0.397913 -1.510058 -1.103121 -1.170753 -1.144196  1.220069   
2005-02-01     -0.139953 -1.570689 -1.369811 -2.093786 -1.582297  1.178063   
2005-03-01      0.056575 -0.357323 -0.470627 -0.333042 -1.036493  1.634883   
2005-04-01     -0.305451  0.447180 -0.068961 -0.228328 -0.079748  0.547967   
2005-05-01     -0.202015  0.808253  1.186897  0.970064  0.996832  0.295928   

                                                       OBS_VALUE_QUANTITY_TON  \
PARTNER_Labels    France   Germany   Hungary   Ukraine                 Brazil   
TIME_PERIOD                                                                     
2005-01-01     -1.411612 -1.450704 -1.797334 -0.943533                    NaN   
2005-02-01     -1.858963 -1.991585 -2.199518 -1.794689              -0.984190   
2005-03-01     -2.318277 -1.583667 -1.675152 -1.121075              -0.984184   
2005-04-01     -0.252798 -0.225310 -0.091872 -0.256108              -0.984179   
2005-05-01      0.336368  0.321316  0.287827  0.706408              -0.984174   

                                                     OBS_VALUE_VALUE_1000EURO  \
PARTNER_Labels    France   Germany   Hungary Ukraine                   Brazil   
TIME_PERIOD                                                                     
2005-01-01      0.455904  0.312128 -0.601425     NaN                      NaN   
2005-02-01      1.056377  2.004749 -0.107375     NaN                -0.866556   
2005-03-01      0.075519  1.633606 -0.177734     NaN                -0.866447   
2005-04-01      1.401945  0.976343 -0.505332     NaN                -0.866338   
2005-05-01      0.925602  0.460284  0.022237     NaN                -0.866229   

                                                       OIL_PRICE $/bbl  \
PARTNER_Labels    France   Germany   Hungary   Ukraine          Brazil   
TIME_PERIOD                                                              
2005-01-01     -0.729229 -0.413099 -0.625766       NaN       -1.267942   
2005-02-01     -0.244912  0.395627 -0.330115       NaN       -1.190552   
2005-03-01     -0.996221  0.354065 -0.384275       NaN       -0.933780   
2005-04-01     -0.152379 -0.151961 -0.574298 -1.033377       -0.946458   
2005-05-01     -0.360411 -0.362265 -0.204267 -1.033376       -1.064429   

                                                       agricultural_land  \
PARTNER_Labels    France   Germany   Hungary   Ukraine            Brazil   
TIME_PERIOD                                                                
2005-01-01     -1.267942 -1.267942 -1.267942 -1.267942         -1.802402   
2005-02-01     -1.190552 -1.190552

#### SAVE EXCEL FILE FILTERED STANDARDIZED

In [85]:
from datetime import datetime
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'MAIZE_FILTERED_STANDARDIZED_{now}.xlsx'
df_maize_total_filtered_std.to_excel(filename, index=True)

# CODE JUNK BELOW

In [ ]:
# Filter the data to include only the desired countries
#df_filtered = df_merged[df_merged['PARTNER_Labels'].isin(top_partners)]

# Create the pivot table
#df_filtered_maize_total = df_filtered.pivot_table(index='TIME_PERIOD',columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total'])
#df_filtered_maize_total

In [ ]:
df_filtered_maize_corn_trade_weather_futures.to_csv("presentforGonem.csv", index=True)

In [ ]:
df_maize_total_filtered.to_excel("maize_23220231904.xlsx")

In [ ]:
df_maize_total_filtered['price'].median()

In [ ]:
# get the current date and time
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# save the DataFrame to an Excel file with the current timestamp in the file name
filename = f'data_{now}.xlsx'
df.to_excel(filename, index=False)